In [37]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from util_visualization import plot_feature_importances, plot_roc_curve
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, mean_absolute_percentage_error, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import roc_auc_score, roc_curve
from pylab import rcParams
from imblearn.over_sampling import SMOTE
from sklearn.impute import SimpleImputer
from sklearn.utils.class_weight import compute_class_weight
from sklearn.utils import class_weight

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
import pickle
import importlib
import sys
import joblib
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import glob
import math

#For NN
from keras.models import Sequential
from tensorflow import keras
from keras import backend as K
import tensorflow_addons as tfa
from keras.layers import Dense, LSTM, BatchNormalization, RepeatVector, TimeDistributed, Dropout
from keras.optimizers import Adam, RMSprop, Adagrad, Nadam
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.regularizers import l2
from time import time
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

## Import Data

In [13]:
# Load the labeled index data from csv
labeled_index = pd.read_csv("../data/labeled_data/lagged_labeled_data.csv")
labeled_index.set_index(labeled_index['date'], inplace=True)
quarters = np.sort(labeled_index.index.unique())

#Features Columns
features_columns = list(labeled_index.columns)
columns_to_remove = ['index', 'crash_label', 'date', 'Quarter']
features_columns = [column for column in features_columns if column not in columns_to_remove]
print(features_columns)

['000001.SS', '^BSESN', '^BVSP', '^FCHI', '^FTLC', '^GDAXI', '^GSPC', '^HSI', '^N225', '^SSMI', 'volatility_lag1', 'volatility_lag2', 'price_change_lag1', 'price_change_lag2', 'volume_change_lag1', 'volume_change_lag2', 'Crude_Oil_Index_Excess_Return_lag1', 'Crude_Oil_Index_Excess_Return_lag2', 'Ted_Rate_lag1', 'Ted_Rate_lag2', 'Gold_Price_lag1', 'Gold_Price_lag2', 'housing_lag1', 'housing_lag2', 'reserve_lag1', 'reserve_lag2', '10YGov_BondYield_lag1', '10YGov_BondYield_lag2', 'current_acct_lag1', 'current_acct_lag2', 'FX_Rate_lag1', 'FX_Rate_lag2', 'turnover_lag1', 'turnover_lag2', 'Population_lag1', 'Population_lag2', 'npl_lag1', 'npl_lag2', 'Recession_Indicators_lag1', 'Recession_Indicators_lag2', 'inflation_lag1', 'inflation_lag2', 'Unemployment_lag1', 'Unemployment_lag2', 'GDP_lag1', 'GDP_lag2']


In [51]:
# Selected by eliminating 30% 0f features with lowest abs importance_coed in log regression 
features_columns =['000001.SS', '^BSESN', '^BVSP', '^FCHI', '^FTLC', '^GDAXI', '^GSPC', '^HSI', '^N225', '^SSMI',
 'current_acct_lag2', 'npl_lag2', 'npl_lag1', 'Population_lag2',
       'Recession_Indicators_lag1', 'Population_lag1', 'volatility_lag1',
       'volatility_lag2', 'current_acct_lag1', 'turnover_lag1',
       'volume_change_lag2', 'Recession_Indicators_lag2', 'turnover_lag2',
       'Unemployment_lag2', '10YGov_BondYield_lag2',
       '10YGov_BondYield_lag1', 'Unemployment_lag1', 'Ted_Rate_lag1',
       'GDP_lag2', 'inflation_lag2', 'GDP_lag1',
       'Crude_Oil_Index_Excess_Return_lag2',
       'Crude_Oil_Index_Excess_Return_lag1', 'housing_lag2',
       'price_change_lag1', 'inflation_lag1']

## Split data

In [52]:
train_index = math.ceil(len(quarters) * .7)
val_index = math.ceil(len(quarters) * .85)
train_quarters, val_quarters, test_quarters = quarters[:train_index], quarters[train_index:val_index], quarters[val_index:]
train = labeled_index.loc[train_quarters]
val = labeled_index.loc[val_quarters]
test = labeled_index.loc[test_quarters]
X_train = train[features_columns]
y_train = train['crash_label']
X_val = val[features_columns]
y_val = val['crash_label']
X_test = test[features_columns]
y_test = test['crash_label']

print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(610, 36) (610,) (130, 36) (130,) (120, 36) (120,)


## Oversampling

In [53]:
# Oversample the minority class (1) using SMOTE
oversampler = SMOTE(sampling_strategy=0.7)
X_train_resampled, y_train_resampled = oversampler.fit_resample(X_train, y_train)
print(X_train_resampled.shape, y_train_resampled.shape)

(901, 36) (901,)


In [54]:
crash_count = y_train_resampled.value_counts()[1]
crash_percen = crash_count / y_train_resampled.shape[0]
print("Oversampled crash percentage: ", crash_percen)

Oversampled crash percentage:  0.4117647058823529


## Reshape

In [55]:
""" Changes data to the format for LSTM training 
for sliding window approach 
Goal: capture temporal dependencies in the data """
# Prepare the list for the transformed data

def lstm_transform(X_data, y_data, num_steps=5):
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(X_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= X_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = np.array(X_data)[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = np.array(y_data)[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_transformed = np.array(X)
    y_transformed = np.array(y)
    return x_transformed, y_transformed

In [56]:
X_train_reshaped, y_train_reshaped = lstm_transform(X_train_resampled, y_train_resampled)

X_val_reshaped, y_val_reshaped = lstm_transform(X_val, y_val)

X_test_reshaped, y_test_reshaped = lstm_transform(X_test, y_test)

In [57]:
print(X_train_reshaped.shape, y_train_reshaped.shape, X_val_reshaped.shape, y_val_reshaped.shape, X_test_reshaped.shape, y_test_reshaped.shape)

(896, 5, 36) (896,) (125, 5, 36) (125,) (115, 5, 36) (115,)


## Modelling

In [20]:
metrics = [
    keras.metrics.FalseNegatives(name="fn"),
    keras.metrics.FalsePositives(name="fp"),
    keras.metrics.TrueNegatives(name="tn"),
    keras.metrics.TruePositives(name="tp"),
    keras.metrics.Precision(name="precision"),
    keras.metrics.Recall(name="recall"),
]

In [58]:
# Many-to-One Training
# X_train_reshaped[1]: number of time steps
# X_train_reshaped[2]: number of features

def create_model(optimizer='adam', learning_rate=0.001):
    if optimizer == 'adam':
        opt = Adam(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = RMSprop(learning_rate=learning_rate)
    elif optimizer == 'adagrad':
        opt = Adagrad(learning_rate=learning_rate)
    elif optimizer == 'nadam':
        opt = Nadam(learning_rate=learning_rate)
    else:
        raise ValueError('Invalid optimizer specified.')
        
    model = Sequential()
    model.add(LSTM(128, dropout=0.2, return_sequences=True, input_shape=(X_train_reshaped.shape[1],X_train_reshaped.shape[2])))
    model.add(Dropout(0.2))  
    model.add(LSTM(64)) 
    model.add(Dense(1, activation='sigmoid'))
    optimizer = Adam(lr=learning_rate)
    model.compile(optimizer=opt, loss='binary_crossentropy', metrics=metrics)
    
    return model

In [22]:
# Hyperparam Tuning: optimizer, learning rate, batch size
tscv = TimeSeriesSplit(n_splits=3)
param_grid = {
    'optimizer': ['adam', 'rmsprop', 'adagrad', 'nadam'],
    'learning_rate': [0.001, 0.01, 0.1],
    'batch_size': [32, 64, 128]
}
lstm_model = KerasClassifier(build_fn=create_model)
grid_search = GridSearchCV(estimator=lstm_model, param_grid=param_grid, cv=tscv)
grid_result = grid_search.fit(X_val_reshaped, y_val_reshaped)

# Get the best hyperparameters
best_learning_rate = grid_result.best_params_['learning_rate']
best_batch_size = grid_result.best_params_['batch_size']
best_optimizer = grid_result.best_params_['optimizer']

print("Best Optimizer:", best_optimizer)
print("Best Learning Rate:", best_learning_rate)
print("Best Batch Size:", best_batch_size)

2024-04-07 01:55:18.933793: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2024-04-07 01:55:20.498810: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-07 01:55:21.035999: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPer

1/1 [==============================] - 3s 3s/step - loss: 0.6028 - fn: 0.0000e+00 - fp: 2.0000 - tn: 30.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00


2024-04-07 01:55:22.321047: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-07 01:55:22.469153: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/1 [==============================] - 1s 841ms/step - loss: 0.6493 - fn: 2.0000 - fp: 5.0000 - tn: 24.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x31x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x31x1x1xi1>'
2024-04-07 01:55:24.377281: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
202

...

1/1 [==============================] - 1s 1s/step - loss: 0.3099 - fn: 0.0000e+00 - fp: 4.0000 - tn: 27.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x31x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x31x1x1xi1>'
2024-04-07 01:56:52.888895: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
202

3/4 [=====================>........] - ETA: 0s - loss: 0.7232 - fn: 3.0000 - fp: 59.0000 - tn: 61.0000 - tp: 4.0000 - precision: 0.0635 - recall: 0.5714

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x29x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x29x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceSh

4/4 [==============================] - 3s 148ms/step - loss: 0.7048 - fn: 3.0000 - fp: 67.0000 - tn: 78.0000 - tp: 8.0000 - precision: 0.1067 - recall: 0.7273
Best Optimizer: adam
Best Learning Rate: 0.001
Best Batch Size: 32


In [59]:
model = create_model(best_optimizer, best_learning_rate)

model.summary()

Model: "sequential_32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_34 (LSTM)              (None, 5, 128)            84480     
                                                                 
 dropout_3 (Dropout)         (None, 5, 128)            0         
                                                                 
 lstm_35 (LSTM)              (None, 64)                49408     
                                                                 
 dense_31 (Dense)            (None, 1)                 65        
                                                                 
Total params: 133,953
Trainable params: 133,953
Non-trainable params: 0
_________________________________________________________________


In [60]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                classes=np.unique(y_train_reshaped), 
                y=y_train_reshaped)
train_class_weights = dict(enumerate(class_weights))
print(train_class_weights)

{0: 0.8468809073724007, 1: 1.220708446866485}


In [61]:
model_result = model.fit(X_train_reshaped, y_train_reshaped, batch_size=best_batch_size, epochs=50, validation_data=(X_val_reshaped, y_val_reshaped), class_weight=train_class_weights)

Epoch 1/50


2024-04-07 02:14:01.106518: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-07 02:14:01.489421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-07 02:14:01.604404: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Cac

28/28 [==============================] - ETA: 0s - loss: 0.5343 - fn: 57.0000 - fp: 192.0000 - tn: 451.0000 - tp: 311.0000 - precision: 0.6183 - recall: 0.8451

2024-04-07 02:14:04.410482: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-07 02:14:04.582641: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-07 02:14:04.739066: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x32x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Cac

28/28 [==============================] - 6s 114ms/step - loss: 0.5343 - fn: 57.0000 - fp: 192.0000 - tn: 451.0000 - tp: 311.0000 - precision: 0.6183 - recall: 0.8451 - val_loss: 0.7463 - val_fn: 9.0000 - val_fp: 40.0000 - val_tn: 74.0000 - val_tp: 2.0000 - val_precision: 0.0476 - val_recall: 0.1818
Epoch 2/50
 1/28 [>.............................] - ETA: 0s - loss: 0.6163 - fn: 1.0000 - fp: 7.0000 - tn: 12.0000 - tp: 12.0000 - precision: 0.6316 - recall: 0.9231

loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x29x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x29x1x1xi1>'


28/28 [==============================] - 1s 36ms/step - loss: 0.4101 - fn: 64.0000 - fp: 81.0000 - tn: 448.0000 - tp: 303.0000 - precision: 0.7891 - recall: 0.8256 - val_loss: 0.5916 - val_fn: 10.0000 - val_fp: 22.0000 - val_tn: 92.0000 - val_tp: 1.0000 - val_precision: 0.0435 - val_recall: 0.0909
Epoch 3/50
28/28 [==============================] - 1s 35ms/step - loss: 0.3602 - fn: 58.0000 - fp: 65.0000 - tn: 464.0000 - tp: 309.0000 - precision: 0.8262 - recall: 0.8420 - val_loss: 0.6106 - val_fn: 10.0000 - val_fp: 27.0000 - val_tn: 87.0000 - val_tp: 1.0000 - val_precision: 0.0357 - val_recall: 0.0909
Epoch 4/50
28/28 [==============================] - 1s 31ms/step - loss: 0.3242 - fn: 52.0000 - fp: 53.0000 - tn: 476.0000 - tp: 315.0000 - precision: 0.8560 - recall: 0.8583 - val_loss: 0.6955 - val_fn: 9.0000 - val_fp: 30.0000 - val_tn: 84.0000 - val_tp: 2.0000 - val_precision: 0.0625 - val_recall: 0.1818
Epoch 5/50
28/28 [==============================] - 1s 32ms/step - loss: 0.2903 - 

28/28 [==============================] - 1s 31ms/step - loss: 0.0807 - fn: 15.0000 - fp: 14.0000 - tn: 515.0000 - tp: 352.0000 - precision: 0.9617 - recall: 0.9591 - val_loss: 1.5107 - val_fn: 9.0000 - val_fp: 34.0000 - val_tn: 80.0000 - val_tp: 2.0000 - val_precision: 0.0556 - val_recall: 0.1818
Epoch 29/50
28/28 [==============================] - 1s 30ms/step - loss: 0.0731 - fn: 14.0000 - fp: 10.0000 - tn: 519.0000 - tp: 353.0000 - precision: 0.9725 - recall: 0.9619 - val_loss: 1.5891 - val_fn: 9.0000 - val_fp: 37.0000 - val_tn: 77.0000 - val_tp: 2.0000 - val_precision: 0.0513 - val_recall: 0.1818
Epoch 30/50
28/28 [==============================] - 1s 30ms/step - loss: 0.0733 - fn: 14.0000 - fp: 12.0000 - tn: 517.0000 - tp: 353.0000 - precision: 0.9671 - recall: 0.9619 - val_loss: 1.6208 - val_fn: 9.0000 - val_fp: 34.0000 - val_tn: 80.0000 - val_tp: 2.0000 - val_precision: 0.0556 - val_recall: 0.1818
Epoch 31/50
28/28 [==============================] - 1s 30ms/step - loss: 0.0576 -

In [62]:
results = model.evaluate(X_test_reshaped, y_test_reshaped)
results

4/4 [==============================] - 0s 118ms/step - loss: 0.4020 - fn: 1.0000 - fp: 14.0000 - tn: 100.0000 - tp: 0.0000e+00 - precision: 0.0000e+00 - recall: 0.0000e+00


loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x19x1x1xi1>'
loc("mps_select"("(mpsFileLoc): /AppleInternal/Library/BuildRoots/0032d1ee-80fd-11ee-8227-6aecfccc70fe/Library/Caches/com.apple.xbs/Sources/MetalPerformanceShadersGraph/mpsgraph/MetalPerformanceShadersGraph/Core/Files/MPSGraphUtilities.mm":294:0)): error: 'anec.gain_offset_control' op result #0 must be 4D/5D memref of 16-bit float or 8-bit signed integer or 8-bit unsigned integer values, but got 'memref<1x19x1x1xi1>'


[0.40203744173049927, 1.0, 14.0, 100.0, 0.0, 0.0, 0.0]

In [63]:
y_prob = model.predict(X_test_reshaped)
y_pred = np.where(y_prob > 0.5, 1, 0) # Adjust the threshold as needed
accuracy = accuracy_score(y_test_reshaped.flatten(), y_pred.flatten())
rmse = np.sqrt(mean_squared_error(y_test_reshaped.flatten(), y_pred.flatten()))
mape = np.mean(np.abs((y_test_reshaped.flatten() - y_pred.flatten()) / y_test_reshaped.flatten())) * 100

conf_matrix = confusion_matrix(y_test_reshaped.flatten(), y_pred.flatten(), labels=[0, 1])

precision = precision_score(y_test_reshaped.flatten(), y_pred.flatten())
recall = recall_score(y_test_reshaped.flatten(), y_pred.flatten())
f1 = f1_score(y_test_reshaped.flatten(), y_pred.flatten())
fpr, tpr, thresholds = roc_curve(y_test_reshaped.flatten(), y_prob.flatten())
auc_roc = roc_auc_score(y_test_reshaped.flatten(), y_prob)

2024-04-07 02:14:50.080247: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2024-04-07 02:14:50.154852: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


1/4 [======>.......................] - ETA: 3s

2024-04-07 02:14:50.305213: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


4/4 [==============================] - 1s 46ms/step


## Evaluation

In [64]:
evaluation_result = {
    'Train_Start': pd.to_datetime(train_quarters).min(),
    'Train_End': pd.to_datetime(train_quarters).max(),
    'Val_Start': pd.to_datetime(val_quarters).min(),
    'Val_End': pd.to_datetime(val_quarters).max(),
    'Test_Start': pd.to_datetime(test_quarters).min(),
    'Test_End': pd.to_datetime(test_quarters).max(),
    'Confusion_Matrix': conf_matrix, #[[TN, FP],[FN,TP]]
    'Precision': precision, 
    'Recall': recall, 
    'F1': f1, 
    'Accuracy': accuracy, 
    'RMSE': rmse,
    'MAPE': mape,
    'AUC-ROC': auc_roc,
    'False Positive Rate': fpr,
    'True Positive Rate': tpr
    }
evaluation_result

{'Train_Start': Timestamp('1998-09-30 00:00:00'),
 'Train_End': Timestamp('2013-09-30 00:00:00'),
 'Val_Start': Timestamp('2013-12-31 00:00:00'),
 'Val_End': Timestamp('2016-12-31 00:00:00'),
 'Test_Start': Timestamp('2017-03-31 00:00:00'),
 'Test_End': Timestamp('2019-12-31 00:00:00'),
 'Confusion_Matrix': array([[100,  14],
        [  1,   0]]),
 'Precision': 0.0,
 'Recall': 0.0,
 'F1': 0.0,
 'Accuracy': 0.8695652173913043,
 'RMSE': 0.3611575592573076,
 'MAPE': nan,
 'AUC-ROC': 0.8245614035087719,
 'False Positive Rate': array([0.        , 0.00877193, 0.1754386 , 0.1754386 , 1.        ]),
 'True Positive Rate': array([0., 0., 0., 1., 1.])}

In [ ]:
evaluation_df = pd.DataFrame(evaluation_result)
evaluation_df.to_csv('./evaluation_result/' + f"evaluation_LSTM_features.csv", index=False)
evaluation_df

TODO: https://stackoverflow.com/questions/77032200/how-can-i-correct-my-time-series-lstm-rnn-for-binary-classification-favoring-cla